In [1]:
import pylab
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table

In [2]:
import seaborn as sns

This notebook uses all the raw data from the XID+MIPS catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../data/ELAIS-S1/holes_ELAIS-S1_irac1_O16_20180122_MOC.fits')


## Read in XID+MIPS catalogue

In [4]:
XID_MIPS=Table.read('../dmu26_XID+MIPS_ELAIS-S1/data/dmu26_XID+MIPS_ELAIS-S1_20180215.fits')

In [5]:
XID_MIPS[0:10]

help_id,RA,Dec,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24
,deg,deg,uJy,uJy,uJy,MJy / sr,MJy / sr,,,,
str27,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,bool
HELP_J003311.854-452559.212,8.2993897334,-45.4331144811,70.8551,100.277,43.0187,-0.00853529,4.94265e-06,0.998633,1508.0,0.0,False
HELP_J003311.199-452603.296,8.2966614384,-45.4342490173,136.227,164.495,106.93,-0.00853529,4.94265e-06,1.00005,2000.0,0.0,False
HELP_J003315.424-452615.143,8.31426712989,-45.4375397927,93.2529,124.365,62.9979,-0.00853529,4.94265e-06,nan,1138.0,0.0,False
HELP_J003315.452-452606.430,8.31438135339,-45.4351194503,9.39005,23.3159,2.61225,-0.00853529,4.94265e-06,nan,2000.0,0.0,True
HELP_J003314.599-452546.572,8.31082943334,-45.4296033101,152.254,179.911,125.606,-0.00853529,4.94265e-06,nan,2000.0,0.0,False
HELP_J003306.225-452506.589,8.27593562562,-45.4184968362,372.777,395.828,348.451,-0.00271362,5.25661e-06,nan,2000.0,0.0,False
HELP_J003302.815-452550.432,8.26173107575,-45.4306756516,12.826,28.9045,3.70633,-0.00271362,5.25661e-06,nan,2000.0,0.0,True
HELP_J003304.883-452634.388,8.27034568672,-45.4428854649,430.479,459.679,402.075,-0.00271362,5.25661e-06,nan,2000.0,0.0,False


The uncertianties become Gaussian by $\sim 20 \mathrm{\mu Jy}$

In [6]:
good=XID_MIPS['F_MIPS_24']>30

In [7]:
good.sum()

194276

## Read in Maps

In [9]:

pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/ELAIS-S1_SPIRE250_v0.9.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/ELAIS-S1_SPIRE350_v0.9.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/ELAIS-S1_SPIRE500_v0.9.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [10]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

In [11]:
## Set XID+ prior class

In [12]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(XID_MIPS['RA'][good],XID_MIPS['Dec'][good],'dmu26_XID+MIPS_ELAIS-S1_20180215.fits',ID=XID_MIPS['help_id'][good])#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(XID_MIPS['RA'][good],XID_MIPS['Dec'][good],'dmu26_XID+MIPS_ELAIS-S1_20180215.fits',ID=XID_MIPS['help_id'][good])
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(XID_MIPS['RA'][good],XID_MIPS['Dec'][good],'dmu26_XID+MIPS_ELAIS-S1_20180215.fits',ID=XID_MIPS['help_id'][good])
prior500.prior_bkg(-5.0,5)

In [13]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [14]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 580 tiles required for input catalogue and 18 large tiles
writing total_bytes=225838061...
writing bytes [0, 225838061)... done.


SystemExit: 

In [14]:
prior250.nsrc

194276